# Data analysis of the USA commercial flights arrival and departure details from 2003 to 2004

## Preamble: overview of the dataset

### First, we load the raw data and import the libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from matplotlib.gridspec import GridSpec
from matplotlib import rcParams

import plotly.graph_objects as go

from sklearn import metrics, linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from scipy.optimize import curve_fit
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last_expr"
pd.options.display.max_columns = 50
%matplotlib inline

rename_week= {1:'Monday',2:'Tuesday',3:'Wednesday',4:'Thursday',5:'Friday',6:'Saturday',7:'Sunday'}
rename_month={1:"January",2:"February",3:"March",4:"April",5:"May",6:"June",7:"July",8:"August",9:"September",10:"October",11:"November",12:"December"}

In [ ]:
# Load flight_delays_2003 file
flight_2003 = pd.read_csv("C:/Users/Administrator/Desktop/2003.csv",na_values=['-'])  #missing values are marked '-'

# Load flight_2004 file
flight_2004 = pd.read_csv("C:/Users/Administrator/Desktop/2004.csv",na_values=['-'])  #missing values are marked '-'

In [ ]:
# Combine the 2 dataframes using append() function
df = flight_2003.append(flight_2004)
display(df)

In [ ]:
# Load plane-data file
planes = pd.read_csv("C:/Users/Administrator/Desktop/plane-data.csv",na_values=['-'])  #missing values are marked '-'
planes = planes[['tailnum','year']]

planes

In [ ]:
# join planes with dataframe
planes=planes[planes['year'].isnull()==False]
df = df.merge(planes[['tailnum','year']],how='left',left_on='TailNum',right_on='tailnum')
df.head()

In [ ]:
airports = pd.read_csv("C:/Users/Administrator/Desktop/airports.csv")
airports=airports[['iata','airport','city','state','country','lat','long']].drop_duplicates().reset_index(drop=True)
airports

In [ ]:
# join airports with dataframe
df = (df.merge(airports, left_on='Origin', right_on='iata'))
df.head()

### Data Cleaning

In [ ]:
main_df = df[df["Cancelled"]==0]

In [ ]:
# View information about each column
main_df.info()

In [ ]:
# Calculate the number of missing columns
main_df.isnull().sum()

In [ ]:
main_df[main_df.isnull().values.any(axis=1)].head()

In [ ]:
main_df['If_Delayed_Dep'] = main_df['DepDelay'].apply(lambda x:x > 0)
main_df['If_Delayed_Arr'] = main_df['ArrDelay'].apply(lambda x:x > 0)

main_df=main_df[main_df['DepTime'].notna()]
main_df=main_df[main_df['DepDelay'].notna()]
main_df=main_df[main_df['ArrDelay'].notna()]
# View the dimensions of the data
main_df.shape

In [ ]:
print(main_df.head())

## 1. The best time of day, day of the week, and time of year to fly to minimise delays

### 1.1 The best time of day to fly to minimise delays

In [ ]:
#Screen out the data of all flights with Departure delays，i.e.DepDelay>0
df=main_df['DepDelay']>0
filtered_df1 = main_df[df]
filtered_df1.head()

In [ ]:
def applyDayTime(x):
    if len(x)==3:
        b='0'
        c=b+x
        return c[:2]
    else:
        return x[:2]

def getTimeofDay(x):
    if int(x) <= 6:
        return "Early Morning"
    elif int(x) > 6 and int(x) <= 12:
        return "Late Morning"
    elif int(x) > 12 and int(x) <=18:
        return "Afternoon"
    elif int(x) > 18:
        return "Evening"

In [ ]:
filtered_df1['DepTime']=filtered_df1['DepTime'].apply(lambda x: str(int(x)))
filtered_df1['DepTime_hour']=filtered_df1['DepTime'].apply(applyDayTime)

In [ ]:
filtered_df1=filtered_df1[filtered_df1['DepTime_hour'].isin(['01','02','03','04','05','06','07','08','09','10','11',
                                                             '12','13','14','15','16','17','18','19','20','21','22','23','24'])]

In [ ]:
filtered_df1['timeofDay']= filtered_df1['DepTime_hour'].apply(getTimeofDay)
filtered_df1.head()

In [ ]:
grouped_hour = filtered_df1.groupby(filtered_df1['timeofDay']) 
grouped_hour = grouped_hour[['timeofDay','If_Delayed_Dep']]
#Sum of grouped data
grouped_hour_sum = grouped_hour.sum()
print(grouped_hour_sum)

In [ ]:
fig,axes = plt.subplots()
grouped_hour_sum['If_Delayed_Dep'].plot(kind='pie',ax=axes,autopct='%.2f%%',radius=2.2,pctdistance=0.6,labeldistance=1.08,
                            textprops={'fontsize':12, 'color':'black'},explode = (0,0.3,0,0),
                            colors=['#FFD700','#EE4000','#9F79EE','#4876FF'],
                            title = 'Depature Delay comparison grouped by time of day')
#The autopct parameter specifies how to display data labels in a pie chart
#'%.2f%%'indicates that the data label is formatted as a percentage with two decimal digits reserved

#Set the aspect ratios of pie charts to be equal
axes.set_aspect('equal')

from matplotlib import rcParams
rcParams['axes.titlepad'] = 110

### 1.2 The best day of the week to fly to minimise delays

In [ ]:
main_df["DayOfWeek"]=main_df["DayOfWeek"].map(rename_week)

delayed_dayofweek = main_df[['DayOfWeek','DepDelay']].groupby('DayOfWeek').agg(['mean','count'])
delayed_dayofweek.columns = ['mean','count']
# SO FROM THIS Saturday has the minimun value for the delay
delayed_dayofweek

In [ ]:
import plotly.express as px
fig = px.bar(delayed_dayofweek, x=delayed_dayofweek.index, y='mean', color='count', height=400, width=700, title="Delays for each day of the week")
fig.show()

### 1.3 The best time of the year to fly to minimise delays

In [ ]:
delayed_data = main_df['If_Delayed_Dep'].value_counts()

In [ ]:
#Delay ratio
print(delayed_data[1] / (delayed_data[0] + delayed_data[1]) )

In [ ]:
main_df["Month"]=main_df["Month"].map(rename_month)

In [ ]:
#Monthly delay, two-level index:'Month','Delayed'
delayed_groupbymonth = main_df.groupby(['Month','If_Delayed_Dep'])
delayed_groupbymonth=delayed_groupbymonth[['Month','DepDelay']]

In [ ]:
#unstack
delayed_groupbymonth = delayed_groupbymonth.size().unstack()
print(delayed_groupbymonth)

In [ ]:
delayed_groupbymonth.plot(xlabel='Month', ylabel='Number of Delay', figsize = [10,6])

In [ ]:
delayed_groupbymonth.plot(kind = 'barh',stacked = True, figsize = [10,6],colormap = 'summer')

In [ ]:
# We find that September is the best month to fly which can minimise the delays
# Let's analyse each day of September to find the best day of the year for the flight to fly to minimise delays
df = main_df[['Year', 'Month', 'DayofMonth','If_Delayed_Dep']]
print(df)

In [ ]:
plt.figure(figsize=(8, 5))
df_September = df[(df.Month == 'September')]['DayofMonth'].value_counts()
sns.barplot(df_September.index, df_September.values)

## 2. Do older planes suffer more delays?

In [ ]:
planes_delays = main_df[['Year','UniqueCarrier', 'TailNum', 'year','DepDelay','If_Delayed_Dep']]
planes_delays.head()

planes_delays=planes_delays[planes_delays['year'].notna()]
planes_delays.shape
print(planes_delays)

In [ ]:
planes_delays = planes_delays.mask(planes_delays.eq('None')).dropna()
print(planes_delays)

In [ ]:
planes_delays.isnull().sum()

In [ ]:
planes_delays.info()

In [ ]:
planes_delays["year"] = planes_delays["year"].astype(str).astype(int)
print(planes_delays.dtypes)

In [ ]:
planes_delays["age"] = planes_delays["Year"] - planes_delays["year"]
print(planes_delays)

In [ ]:
planes_delays = planes_delays[~(planes_delays['age'].isin([2003,2004,-4,-3]))]

In [ ]:
planes_delays = planes_delays[['age','DepDelay']].groupby('age').agg(['mean','count'])
planes_delays.columns = ['mean','count']
print(planes_delays)

In [ ]:
plt.figure(figsize=(15, 4))
age = sns.lineplot(x="age", y="mean", data=planes_delays)
plt.xticks([0,2,4,6,8,10,12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,42,44,46,48])
age.set_title("Trend in flight delays with flights age increase",fontsize = 20,pad=20)
age.set_xlabel("Flight Age", fontsize = 15)
age.set_ylabel("Mean of the DepDelay", fontsize = 15)
#Planes delay is same for older and newer planes

## 3. How does the number of people flying between different locations change over time?

In [ ]:
#If you run the Q1 code, you need to run the below code to rename the month and week.
week_dict1= {'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6,'Sunday':7}
main_df["DayOfWeek"]=main_df["DayOfWeek"].map(week_dict1)

month_dict1= {"January":1,"February":2,"March":3,"April":4,"May":5,"June":6,"July":7,"August":8,"September":9,"October":10,"November":11,"December":12}
main_df["Month"]=main_df["Month"].map(month_dict1)

In [ ]:
loc_df = main_df[~(main_df['CancellationCode'].isin([1]))]
loc_df = loc_df[~(loc_df['Diverted'].isin([1]))]

In [ ]:
loc_df=loc_df.groupby(['Year','Month','DayOfWeek','Origin','Dest','DepTime']).size().to_frame('flights').reset_index()
loc_df.head()

In [ ]:
dest_orig_year=loc_df[['Year','Origin','Dest','flights']].groupby(['Year','Origin','Dest']).sum()
dest_orig_year

In [ ]:
dest_orig_week=loc_df[['DayOfWeek','Origin','Dest','flights']].groupby(['DayOfWeek','Origin','Dest']).sum()
dest_orig_week

In [ ]:
loc_rank2003=dest_orig_year.groupby(['Year','Origin','Dest']).size().to_frame('flights').reset_index()
loc_rank2003.head()

In [ ]:
dest_orig_week=loc_df[['DayOfWeek','Origin','Dest','flights']].groupby(['DayOfWeek','Origin','Dest']).sum()
dest_orig_week

## 4. Can you detect cascading failures as delays in one airport create delays in others?

In [ ]:
caf = main_df[['airport', 'Origin', 'Dest','UniqueCarrier','TailNum','LateAircraftDelay','Year','Month','DayofMonth',
               'DayOfWeek','DepTime','DepDelay','ArrTime','ArrDelay','city','state','country','lat','long']]

caf = caf[~(caf['LateAircraftDelay'].isin([0]))]
caf = caf[~(caf['TailNum'].isin([0]))]
caf = caf.mask(caf.eq('None')).dropna()

In [ ]:
caf_count=caf.groupby(['airport']).agg({"TailNum":"count",
                            "DepDelay":["mean","std"],
                            "LateAircraftDelay":["mean","std"],
                            }).reset_index()
caf_count

In [ ]:
caf_df_map = caf.loc[(caf['Year'] == 2003) & (caf['Month'] == 12) & (caf['DayofMonth'] == 12)]
#caf_df

In [ ]:
import plotly.graph_objects as go

import pandas as pd

fig = go.Figure(data=go.Scattergeo(
        lon = caf_df_map['long'],
        lat = caf_df_map['lat'],
        #text = caf_df_map['text'],
        mode = 'markers',
        marker_color = caf_df_map['LateAircraftDelay'],
        ))

fig.update_layout(
        title = 'US airports that affected by the Late Aircraft delay on 12 December, 2003',
        geo_scope='usa',
    )
fig.show()

In [ ]:
# Let's take flight with TailNum 'N612MQ' as an example
caf_df = caf_df_map.loc[caf_df_map['TailNum'] == 'N612MQ']
caf_df = caf_df.sort_values(by=['DepTime'],ascending=[True])
caf_df

## 5. Create a model to predict the delays

### 5.1 Linear Regression Model

In [ ]:
percent_missing=(main_df.isna().mean() *100)

missing_value_df = pd.DataFrame({'column_name': main_df.columns,'percent_missing': percent_missing})
cols_to_drop=missing_value_df[missing_value_df['percent_missing']>20]['column_name'] .tolist()
cols_to_drop                    

In [ ]:
main_df=main_df.drop(cols_to_drop,axis=1)
main_df=main_df.dropna()
main_df.head()

In [ ]:
main_df['totalDelay']=main_df['DepDelay']+main_df['ArrDelay']

In [ ]:
lbl=LabelEncoder()
main_df['UniqueCarrier']=lbl.fit_transform(main_df['UniqueCarrier'])
main_df['DayOfWeek']=lbl.fit_transform(main_df['DayOfWeek'])

In [ ]:
Features=['UniqueCarrier','Diverted','CarrierDelay','WeatherDelay',
'NASDelay','SecurityDelay','LateAircraftDelay','Month','DayOfWeek']
target=['totalDelay']
X = main_df[Features]
y =main_df[target]

In [ ]:
#Divide data into train and test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=101)

In [ ]:
from sklearn.linear_model import LinearRegression

lm=LinearRegression()
lm.fit(X_train,y_train)

In [ ]:
y_pred=lm.predict(X_test)
print('Accuracy on test set is', round(metrics.r2_score(y_test,y_pred),2))   

### 5.2 Random Forest Model

In [ ]:
#Replace NaN with 0
main_df=main_df.drop(cols_to_drop,axis=1)
main_df=main_df.dropna()
main_df.head()

In [ ]:
main_df['CRSDepTime']=main_df['CRSDepTime'].apply(lambda x: str(int(x)))
main_df['CRSDepTime_hour']=main_df['CRSDepTime'].apply(applyDayTime)
main_df=main_df[main_df['CRSDepTime_hour'].isin(['01','02','03','04','05','06',
                                                '07','08','09','10','11','12',
                                                '13','14','15','16','17','18'
                                                '19','20','21','22','23','24'])]

In [ ]:
main_df['DepTime']=main_df['DepTime'].apply(lambda x: str(int(x)))
main_df['DepTime_hour']=main_df['DepTime'].apply(applyDayTime)
main_df=main_df[main_df['DepTime_hour'].isin(['01','02','03','04','05','06',
                                                '07','08','09','10','11','12',
                                                '13','14','15','16','17','18'
                                                '19','20','21','22','23','24'])]

In [ ]:
main_df = main_df[['Year','Month','DayofMonth','DayOfWeek','DepTime_hour','CRSDepTime_hour','ArrTime','ArrDelay','DepDelay',
                       'Origin','Dest','Cancelled','Diverted','CarrierDelay','WeatherDelay','NASDelay','SecurityDelay',
                        'LateAircraftDelay','If_Delayed_Arr','If_Delayed_Dep']]
main_df.head()

In [ ]:
main_df = pd.get_dummies(main_df, columns=['Origin', 'Dest'])
main_df.head()

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(main_df.drop('If_Delayed_Arr', axis=1), main_df['If_Delayed_Arr'],
                                                    test_size=0.2, random_state=42)

In [ ]:
test_x.shape

In [ ]:
model = RandomForestClassifier(random_state=13)
model.fit(train_x, train_y)

In [ ]:
predicted = model.predict(test_x)
model.score(test_x, test_y)

In [ ]:
roc_auc_score(test_y, probabilities[:, 1])

In [ ]:
confusion_matrix(test_y, predicted)

In [ ]:
train_predictions = model.predict(train_x)
precision_score(train_y, train_predictions)

In [ ]:
recall_score(train_y, train_predictions)

In [ ]:
%matplotlib inline
 
sns.set()

In [ ]:
fpr, tpr, _ = roc_curve(test_y, probabilities[:, 1])
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], color='grey', lw=1, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')